## Ngrams lab
LLM's and ChatGPT | Fall 2023 | McSweeney | CUNY Graduate Center

**Due:** September 17


### Background
The purpose of this lab is to explore ngram models. Ngram models are a good introduction to language models generally. Language models are probabilistic representations of language. Ngrams have the benefit of being easy to interrogate and relatively easy to understand (as compared to neural networks).

In this lab, you will build an ngram model from the corpus of your choosing. The example is with 'The Great Gatsby' from Project Gutenberg, but there's a code block for any text file on your computer  


#### Notes
This lab is based heavily on the [nltk documentation](https://www.nltk.org/api/nltk.lm.html)

In [ ]:
import numpy as np
import re

import nltk
# if you haven't downloaded punkt before, you only need to run the line below once
# nltk.download('punkt')
from nltk import word_tokenize
from nltk import sent_tokenize

from nltk.util import bigrams
from nltk.lm.preprocessing import padded_everygram_pipeline

import requests

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Part 1
An example of how ngrams are generated

In [ ]:
# you will need to leverage the requests package
r = requests.get(r'https://www.gutenberg.org/cache/epub/64317/pg64317.txt')
great_gatsby = r.text

# first, remove unwanted new line and tab characters from the text
for char in ["\n", "\r", "\d", "\t"]:
    great_gatsby = great_gatsby.replace(char, " ")

# check
print(great_gatsby[:100])


﻿The Project Gutenberg eBook of The Great Gatsby        This ebook is for the use of anyone anywhere


In [ ]:
# remove the metadata at the beginning - this is slightly different for each book
great_gatsby = great_gatsby[983:]

#### Txt locally
If you'd rather use a file on your computer, here's the code -- you just need to save the text file in your local directory, and change the variables throughout.

The example is a report from the [Congressional Research Service](https://www.everycrsreport.com/files/2020-11-10_R45178_62d6238caecf6c02ddf495be33b3439f09eed744.pdf) on AI and National Security.

In [ ]:
# this is simplified for demonstration
def sample_clean_text(text: str):
    # lowercase
    text = text.lower()

    # remove punctuation from text
    text = re.sub(r"[^\w\s]", "", text)

    # tokenize the text
    tokens = nltk.word_tokenize(text)

    # return your tokens
    return tokens

# call the function
sample_tokens = sample_clean_text(text = great_gatsby)

# check
print(sample_tokens[:50])

['then', 'wear', 'the', 'gold', 'hat', 'if', 'that', 'will', 'move', 'her', 'if', 'you', 'can', 'bounce', 'high', 'bounce', 'for', 'her', 'too', 'till', 'she', 'cry', 'lover', 'goldhatted', 'highbouncing', 'lover', 'i', 'must', 'have', 'you', 'thomas', 'parke', 'dinvilliers', 'i', 'in', 'my', 'younger', 'and', 'more', 'vulnerable', 'years', 'my', 'father', 'gave', 'me', 'some', 'advice', 'that', 'ive', 'been']


In [ ]:
# create bigrams from the sample tokens
my_bigrams = bigrams(sample_tokens)

# check
list(my_bigrams)[:10]

[('then', 'wear'),
 ('wear', 'the'),
 ('the', 'gold'),
 ('gold', 'hat'),
 ('hat', 'if'),
 ('if', 'that'),
 ('that', 'will'),
 ('will', 'move'),
 ('move', 'her'),
 ('her', 'if')]

# Part 2 - creating an ngram model


In [ ]:
# 2 is for bigrams
n = 2
#specify the text you want to use
text = great_gatsby


Now we are going to use an NLTK shortcut for preprocessing. This will:
* pad all of the sentences with `<s>` and `</s>` to train on sentence boundaries, too.
* create both unigrams and bigrams
* create a training set and a full vocab to train on

We need to give it a pre-tokenized text (we'll use nltk's tokenizer)

In [ ]:
# step 1: tokenize the text into sentences
sentences = nltk.sent_tokenize(text)

# step 2: tokenize each sentence into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# step 3: convert each word to lowercase
tokenized_text = [[word.lower() for word in sent] for sent in tokenized_sentences]

#notice the sentence breaks and what the first 10 items of the tokenized text
print(tokenized_text[0])

# commas are still here?

['then', 'wear', 'the', 'gold', 'hat', ',', 'if', 'that', 'will', 'move', 'her', ';', 'if', 'you', 'can', 'bounce', 'high', ',', 'bounce', 'for', 'her', 'too', ',', 'till', 'she', 'cry', '“', 'lover', ',', 'gold-hatted', ',', 'high-bouncing', 'lover', ',', 'i', 'must', 'have', 'you', '!', '”', 'thomas', 'parke', 'd', '’', 'invilliers', 'i', 'in', 'my', 'younger', 'and', 'more', 'vulnerable', 'years', 'my', 'father', 'gave', 'me', 'some', 'advice', 'that', 'i', '’', 've', 'been', 'turning', 'over', 'in', 'my', 'mind', 'ever', 'since', '.']


Why tokenize sentences and words?
We want to be able to retain sentence boundaries to encode that, too.

In [ ]:
# notice what the first 10 items are of the vocabulary
print(text[:100])

# This looks like each "item" is a character, not neccesarily a vocab

 Then wear the gold hat, if that will move her;    If you can bounce high, bounce for her too,    Ti


In [ ]:
# we imported this function from nltk
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

In [ ]:
from nltk.lm import MLE
# we imported this function from nltk linear models (lm)
# it is for Maximum Likelihood Estimation

# MLE is the model we will use
lm = MLE(n)

In [ ]:
# currently the vocab length is 0: it has no prior knowledge
len(lm.vocab)

0

In [ ]:
# fit the model
# training data is the bigrams and unigrams
# the vocab is all the sentence tokens in the corpus

lm.fit(train_data, padded_sents)
len(lm.vocab)

6953

In [ ]:
# inspect the model's vocabulary.
# be sure that a sentence you know exists (from tokenized_text) is in the
print(lm.vocab.lookup(tokenized_text[0]))

('then', 'wear', 'the', 'gold', 'hat', ',', 'if', 'that', 'will', 'move', 'her', ';', 'if', 'you', 'can', 'bounce', 'high', ',', 'bounce', 'for', 'her', 'too', ',', 'till', 'she', 'cry', '“', 'lover', ',', 'gold-hatted', ',', 'high-bouncing', 'lover', ',', 'i', 'must', 'have', 'you', '!', '”', 'thomas', 'parke', 'd', '’', 'invilliers', 'i', 'in', 'my', 'younger', 'and', 'more', 'vulnerable', 'years', 'my', 'father', 'gave', 'me', 'some', 'advice', 'that', 'i', '’', 've', 'been', 'turning', 'over', 'in', 'my', 'mind', 'ever', 'since', '.')


In [ ]:
# see what happens when we include a word that is not in the vocab.
print(lm.vocab.lookup('then wear the gold hat iphone .'.split()))

('then', 'wear', 'the', 'gold', 'hat', '<UNK>', '.')


What did the model replace 'iphone' with?
- """ U N K """



Given that it didn't just return an "out of vocab" error, what does that mean about our model?

In [ ]:
# how many times does daisy appear in the model?
print(lm.counts['daisy'])

# what is the probability of daisy appearing?
# this is technically the relative frequency of daisy appearing
lm.score('daisy')

183


0.0026549057726065954

In [ ]:
# how many times does daisy appear in the model?
print(lm.counts['gold'])

# what is the probability of daisy appearing?
# this is technically the relative frequency of daisy appearing
lm.score('gold')

7


0.00010155377272265665

In [ ]:
# how many times does daisy appear in the model?
print(lm.counts['champagne'])

# what is the probability of daisy appearing?
# this is technically the relative frequency of daisy appearing
lm.score('champagne')

7


0.00010155377272265665

In [ ]:
# how many times does daisy appear in the model?
print(lm.counts['miss'])

# what is the probability of daisy appearing?
# this is technically the relative frequency of daisy appearing
lm.score('miss')

38


0.0005512919090658504

In [49]:
# how many times does daisy appear in the model?
print(lm.counts['love'])

# what is the probability of daisy appearing?
# this is technically the relative frequency of daisy appearing
lm.score('love')

24


0.0003481843636205371

In [50]:
# how many times does daisy appear in the model?
print(lm.counts['dress'])

# what is the probability of daisy appearing?
# this is technically the relative frequency of daisy appearing
lm.score('dress')

15


0.00021761522726283566

In [51]:
# how many times does daisy appear in the model?
print(lm.counts['tears'])

# what is the probability of daisy appearing?
# this is technically the relative frequency of daisy appearing
lm.score('tears')

6


8.704609090513427e-05

In [ ]:
# how often does (daisy, and) occur and what is the relative frequency?
print(lm.counts[['daisy']]['and'])
lm.score('and', 'daisy'.split())

14


0.07650273224043716

In [ ]:
# what is the score of 'UNK'?  --> 0

lm.score("<UNK>")

0.0

Does the relative frequency of 'UNK' change your assumption about how the model behaves?

How should we change our model to account for the fact the `<UNK>` words are not accounted for by the model?

Note: *Programmatically implementing this solution is beyond the scope of this course.*

## Generate text
We want to start our sentence with a word, and use that to predict all the words that come after that. We'll specify how long it should be.

There is a certain amount of randomness encoded into n-gram models. This prevents a model from becoming entirely deterministic. Maximum Likelihood Estimation without some degree of randomness will only produce the most likely result every time. Setting Random Seed means we will get the same result every time.

In [ ]:
# generate a 20 word sentence starting with the word, 'daisy'

print(lm.generate(20, text_seed= 'daisy', random_seed=42))

['other', ',', 'but', 'he', 'turned', 'to', 'their', 'cars', 'blocking', 'the', 'dull', 'light', ',', 'and', 'separated', 'only', 'building', 'in', 'the', 'adventitious']


In [52]:
# generate a 20 word sentence starting with the word, 'daisy'

print(lm.generate(20, text_seed= 'gold', random_seed=34))

['like', 'that', 'was', 'them', 'up', 'at', 'twilight', '.', '</s>', '.', '</s>', '<s>', 'i', 'think', 'so', 'often', ',', 'and', 'gatsby', 'said']


In [65]:
# generate a 20 word sentence starting with the word, 'daisy'

print(lm.generate(20, text_seed= 'Driving', random_seed=21))

['<s>', '“', 'please', 'don', '’', 't', 'they', 'sauntered', 'about', 'gatsby.', '”', 'in', 'popular', '.', '</s>', 'and', 'flowers', ',', '”', '“']


This next code block is just to clean up the tokenized words and make them easier on human eyes. It is literally a detokenizer, which removes some extraneous text markup and reconciles some words back together.

In [66]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(lm, num_words, text_seed, random_seed=42):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in lm.generate(num_words, text_seed=text_seed, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

In [67]:
# Now generate sentences that look much nicer.
generate_sent(lm, 20, text_seed='daisy', random_seed = 42)

'other, but he turned to their cars blocking the dull light, and separated only building in the adventitious'

Try a few more sentences, and try out another text. Once you are satisfied with what ngrams can (and cannot) do - post your code to your Github or another site.

In [ ]:
# Now generate sentences that look much nicer.
generate_sent(lm, 20, text_seed='daisy', random_seed = 42)

In [68]:
# Now generate sentences that look much nicer.
generate_sent(lm, 20, text_seed='gold', random_seed = 42)

'never be cleared up the rain with a little, as it, and separated only building in the adventitious'

In [70]:
# Now generate sentences that look much nicer.
generate_sent(lm, 20, text_seed='your', random_seed = 42)

'other, but he turned to their cars blocking the dull light, and separated only building in the adventitious'

In [73]:
# Now generate sentences that look much nicer.
generate_sent(lm, 20, text_seed='eye', random_seed = 41)

'for a dark trees.'

In [75]:
# Now generate sentences that look much nicer.
generate_sent(lm, 20, text_seed='rain', random_seed = 4)

'bridge, grew more, hard to the sidewalks and newspaper.'